In [1]:
from nocodb_base import fetch_all_rows_from_nocodb, update_row
from file_utils import write_json, write_tsv, read_tsv
import re
import time


Formatted datetime: 20250729111120


In [2]:
# Configuration
xc_token = "KVwjEgUQOwJkjYih9tef5J7A54z05RHliR_1GoSs"
# all cloudfront urls
#table_id = "mxp572bg586eppw"
#view_id = "vwy8ovu97qes5d9h"
# all pages
table_id = "mzmy06jcvenm2no"
view_id = "vw6jzhxqjdvzj3ca"
noco_base_url = "https://tables.rhizome.org"
headers = {
    'accept': 'application/json',
    'xc-token': xc_token,
}
seed_file = "add_keys/.txt"

In [3]:
rows_data = fetch_all_rows_from_nocodb(noco_base_url, headers, table_id, view_id)
#print(rows_data)

test1
Fetching data from offset 0...
Error: 404, {"error":"TABLE_NOT_FOUND","message":"Table 'm4gk39nkz5qkw5v' not found"}
test2
Total posts fetched: 0


In [118]:
from file_utils import write_json, write_tsv
output_file = "aqnb_url_list.txt"
write_tsv(output_file, rows_data)

TSV data written to aqnb_url_list.txt


In [119]:
rows_data = read_tsv("aqnb_url_list.txt")

In [120]:
from urllib.parse import urlparse, urlunparse, quote

def normalize_url(url):
    url = url.strip()
    parsed = urlparse(url)

    scheme = parsed.scheme.lower()
    netloc = parsed.netloc.lower()

    # Lowercase path, strip trailing slash
    path = parsed.path.rstrip('/').lower()

    # Keep query string if present (as-is)
    query = parsed.query

    # Rebuild the normalized URL
    normalized = urlunparse((scheme, netloc, path, '', query, ''))

    # Encode safely (optional but helpful if comparing to encoded index)
    return quote(normalized, safe=':/?&=%')

In [121]:
urls_list = {}
with open(seed_file, 'rt', encoding='utf-8') as f:
    for line in f:
        url = normalize_url(line)
        urls_list[url] = True

In [122]:
def update_row_defined_data(noco_base_url, headers, table_id, record_id, index_status,index_check_field):
    """Update the index_check column in NocoDB."""
    request_url = (
        f"{noco_base_url}/api/v2/tables/{table_id}/records"
    )
    data = {
        "Id": record_id,
        #index_check_field: index_status,
        "type": "tag"
    }
    #print(request_url)
    #print(data)
    try:
        response = requests.patch(request_url, headers=headers, json=data)
        if response.status_code == 200:
            pass
            #print(f"Updated record {record_id} -> index_check: {index_status}")
        else:
            print(f"Failed to update record {record_id}. Status: {response.status_code}")
            print(f"Response text: {response.text}")
    except requests.exceptions.RequestException as e:
        print(f"Error updating record {record_id}: {e}")

In [123]:
import requests

# Process rows and add index status
urls = []
select_field = "new"
new_select_item = "added"

# Build mapping from normalized URL -> row
url_to_row = {}
for row in rows_data:
    url = row.get("URL")
    if not url:
        continue
    normalized = normalize_url(url)
    url_to_row[normalized] = row


for url in urls_list:
    normalized_url_seedlist = normalize_url(url)
    #print(normalized_url_seedlist)

    if normalized_url_seedlist in url_to_row:
        #print("is in table")
        row = url_to_row[normalized_url_seedlist]
        record_id = row.get("Id")
        #print(record_id)
        select_data = row.get(select_field) or ""
        current_items = [item.strip() for item in select_data.split(',') if item.strip()]

        if new_select_item not in current_items:
            current_items.append(new_select_item)
            new_select = ','.join(current_items)
            update_row_defined_data(noco_base_url, headers, table_id, record_id, new_select, select_field)  
        else:
            new_select = current_items
            update_row_defined_data(noco_base_url, headers, table_id, record_id, new_select, select_field)  

    else:
        print("not in table")
        new_select = new_select_item
        request_url = (
            f"{noco_base_url}/api/v2/tables/{table_id}/records"
        )
        data = {
            "URL": normalized_url_seedlist,
            #select_field: new_select,
            "type": "tag"
        }
        
        try:
            response = requests.post(request_url, headers=headers, json=data)
            if response.status_code == 200:
                pass
                print(f"created record -> new value: {normalized_url_seedlist}")
            else:
                print(f"Failed to create. Status: {response.status_code}")
        except requests.exceptions.RequestException as e:
            print(f"Error creating record: {e}")
    

# Save updated rows
#output_file = "cloudfront_not_in_index.json"
output_file = "add_seed.json"

write_json(output_file, urls)
print(f"Updated rows saved to {output_file}")

JSON data written to add_seed.json
Updated rows saved to add_seed.json
